# VAHEAT serial com examples
The following codes are checked based on manual v1.2. See section 'API' control in VAHEAT manual.

https://interherence.com/downloads/

In [42]:
# importing library
import sys
import json
import glob
import time
import serial
import serial.tools.list_ports

In [25]:
# This function will show available serial port.
def get_ports():
    ports = serial.tools.list_ports.comports()
    for port, desc, hwid in sorted(ports):
            print("{}: {} [{}]".format(port, desc, hwid))

In [27]:
get_ports()

COM5: USB シリアル デバイス (COM5) [USB VID:PID=0483:5740 SER= LOCATION=1-1.2]


## Found com port ? Then set them as parameters
If available serial device was found (name is like **''COM5''** in windows), set name and baudrate for VAHEAT. If multiple ports are aveiable, you have to check which port is for VAHEAT. (for example, use device manager in Windows)

In [31]:
############################
# Set the parameters       #
############################
PORT = 'COM5'
BAUDRATE = 115200
TIMEOUT = 1
############################

# Example 1. get_info
The following code will do:
 - set command
 - Open serial port safely (if something wrong within the block, port is correctly closed)
 - Write a command as serial bytes encoded from str by utf-8 via USB cable
 - Read a response from VAHEAT as serial bytes, then decode to str
 - Print the response

In [35]:
#########################
# command of 'get_info' #
#########################
serialCommand = '{"get_info":true}'

with serial.Serial(PORT, BAUDRATE, timeout=TIMEOUT) as vaheat:
    vaheat.write(serialCommand.encode('utf-8'))
    received = vaheat.readall().decode('utf-8')
    vaheat.close()

print(received)

{
  "command": "GET_INFO",
  "success": true,
  "data":
  {
    "serial_number": "VA-SR-0086-1",
    "version": "v1.2.3 (Jul 27 2023 15:57:44)",
    "setpoint_auto": 105.0,
    "power_shock": 20.0,
    "power_direct": 100.0,
    "duration_shock": 1.0
  }
}



### Talk in JSON
As you can see above, you can communicate with VAHEAT in JSON format either data or commands. Conversion from JSON string to dict type of python is easy.

In [36]:
json_dict = json.loads(received)

In [40]:
print(json_dict['success'], json_dict['data'])

True {'serial_number': 'VA-SR-0086-1', 'version': 'v1.2.3 (Jul 27 2023 15:57:44)', 'setpoint_auto': 105.0, 'power_shock': 20.0, 'power_direct': 100.0, 'duration_shock': 1.0}


In [41]:
json_dict

{'command': 'GET_INFO',
 'success': True,
 'data': {'serial_number': 'VA-SR-0086-1',
  'version': 'v1.2.3 (Jul 27 2023 15:57:44)',
  'setpoint_auto': 105.0,
  'power_shock': 20.0,
  'power_direct': 100.0,
  'duration_shock': 1.0}}

# Example 2. Start heating !
The most basic command would be starting and stopping heating.
This example will start heating, wait a while, then stop heating.

In [46]:
#########################
# command of 'start' #
#########################
start_heating = '{ "start_heating": { "mode": "auto", "temperature": 37.0 }}'
stop_heating  = '{ "stop_heating": true }'

with serial.Serial(PORT, BAUDRATE, timeout=TIMEOUT) as vaheat:
    # Start heating
    vaheat.write(start_heating.encode('utf-8'))
    received = vaheat.readall().decode('utf-8')
    json_dict = json.loads(received)
    if json_dict['success'] :
        print("Heating started.")

    # Wait a while
    time.sleep(2)

    # Stop heating
    vaheat.write(stop_heating.encode('utf-8'))
    received = vaheat.readall().decode('utf-8')
    json_dict = json.loads(received)
    if json_dict['success'] :
        print("Heating stopped.")
    
    vaheat.close()

Heating started.
Heating stopped.
